In [9]:
# Functions & loops
# Analyze FFR data
# Add legends to plots
# Create plot showing unclipped rivers and WDPA for a country/continent
# Calculate new DOR for FFR & identify rivers whose DOR becomes > 5, 10, 20 w proposed dams -***How do we recalculate DOR?**

In [10]:
# Imports
import warnings
import os
import sys

import numpy as np
import numpy.ma as ma
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import geopandas as gpd
from geopandas import GeoDataFrame as gdf
from geopandas import GeoSeries as gs
from shapely.geometry import Point, Polygon

import contextily as ctx
import earthpy as et
import earthpy.plot as ep

In [11]:
# Check path and set working directory.
wd_path = os.path.join(et.io.HOME, 'earth-analytics', 'data')
if os.path.exists(wd_path):
    os.chdir(wd_path)
else:
    print("Path does not exist")

In [12]:
# Download data stored on figshare
# NOTE: WDPA, country/continent borders, and ISO csv were downloaded directly to the hub

# Free flowing rivers dataset
et.data.get_data(url="https://ndownloader.figshare.com/files/15090536")

# Future dams
et.data.get_data(url="https://ndownloader.figshare.com/files/22486157")

# Ramsar Sites
et.data.get_data(url="https://ndownloader.figshare.com/files/22507082")

'/home/jovyan/earth-analytics/data/earthpy-downloads/ramsar-site-data'

In [ ]:
# Open the shapefiles with geopandas
wdpa_africa_polys = gpd.read_file(os.path.join(wd_path,
                                               "earthpy-downloads", "WDPA_Africa", "WDPA_Africa.shp"))

# wdpa_na_polys = gpd.read_file(os.path.join(wd_path,
#                                            "earthpy-downloads", "WDPA_N_America", "WDPA_N_America.shp"))

wdpa_sa_polys = gpd.read_file(os.path.join(wd_path,
                                           "earthpy-downloads", "WDPA_S_America", "WDPA_S_America.shp"))

wdpa_europe_polys = gpd.read_file(os.path.join(wd_path,
                                               "earthpy-downloads", "WDPA_Europe", "WDPA_Europe.shp"))

wdpa_asia_polys = gpd.read_file(os.path.join(wd_path,
                                             "earthpy-downloads", "WDPA_Asia", "WDPA_Asia.shp"))

wdpa_aus_polys = gpd.read_file(os.path.join(wd_path,
                                            "earthpy-downloads", "WDPA_Australia", "WDPA_Australia.shp"))

ffr_0to5 = gpd.read_file(os.path.join(wd_path,
                                      "earthpy-downloads", "FFR_DOR_shapefiles", "FFR_DOR_0to5.shp"))

ffr_5to10 = gpd.read_file(os.path.join(wd_path,
                                       "earthpy-downloads", "FFR_DOR_shapefiles", "FFR_DOR_5to10.shp"))

ffr_10to20 = gpd.read_file(os.path.join(wd_path,
                                        "earthpy-downloads", "FFR_DOR_shapefiles", "FFR_DOR_10to20.shp"))

ffr_20plus = gpd.read_file(os.path.join(wd_path,
                                        "earthpy-downloads", "FFR_DOR_shapefiles", "FFR_DOR_20plus.shp"))

ramsar_polys = gpd.read_file(os.path.join(
    "earthpy-downloads", "ramsar-site-data", "ramsar-boundaries",
    "features_publishedPolygon.shp"))

country_borders = gpd.read_file(os.path.join(wd_path, "earthpy-downloads", "country-borders",
                                             "99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp"))

africa_borders = gpd.read_file(os.path.join(wd_path, "earthpy-downloads", "continent-borders",
                                            "Africa.shp"))

continent_iso = pd.read_csv(os.path.join(wd_path, "earthpy-downloads", "continent-borders",
                                            "continent_country.csv"))

# # Open the dams csv files with pandas
# dams_source_path = os.path.join("earthpy-downloads", "future_dams_2015.csv")
# dams_df = pd.read_csv(dams_source_path)

# # Covert the pandas dataframe to a shapefile for plotting and set output path
# dams_path = os.path.join('fhred-proposed-dams')
# if not os.path.exists(dams_path):
#     os.mkdir(dams_path)

# # Define the geometry for the dam points
# geometry = [Point(xy) for xy in zip(dams_df.Lon_Cleaned, dams_df.LAT_cleaned)]
# crs = {'init': 'epsg:4087'}
# geo_df = gdf(dams_df, crs=crs, geometry=geometry)
# geo_df.to_file(driver='ESRI Shapefile', filename=os.path.join(
#     dams_path, 'proposed_dams.shp'))

# # Open the proposed dams shapefile with geopandas
# proposed_dams = gpd.read_file(os.path.join(dams_path, "proposed_dams.shp"))

In [ ]:
# Data cleaning - pull only the columns that we need from each gdf for clarity & to save processing time
# NOTE: could only find RIV_ORD not RIV_CLASS as a column name

# proposed_dams = proposed_dams[['Country',
#                                'Continent', 'Major Basi', 'Stage', 'geometry']]

ramsar_polys = ramsar_polys[['iso3', 'country_en', 'geometry']]

wdpa_africa_polys = wdpa_africa_polys[[
    'NAME', 'DESIG', 'PARENT_ISO', 'geometry']]

# wdpa_na_polys = wdpa_africa_polys[[
#     'NAME', 'DESIG', 'PARENT_ISO', 'geometry']]

wdpa_sa_polys = wdpa_africa_polys[[
    'NAME', 'DESIG', 'PARENT_ISO', 'geometry']]

wdpa_asia_polys = wdpa_africa_polys[[
    'NAME', 'DESIG', 'PARENT_ISO', 'geometry']]

wdpa_europe_polys = wdpa_africa_polys[[
    'NAME', 'DESIG', 'PARENT_ISO', 'geometry']]

wdpa_australia_polys = wdpa_africa_polys[[
    'NAME', 'DESIG', 'PARENT_ISO', 'geometry']]

ffr_0to5 = ffr_0to5[['CONTINENT', 'COUNTRY', 'DOR',
                     'BAS_NAME', 'LENGTH_KM', 'RIV_ORD', 'geometry']]
ffr_5to10 = ffr_5to10[['CONTINENT', 'COUNTRY', 'DOR',
                       'BAS_NAME', 'LENGTH_KM', 'RIV_ORD', 'geometry']]
ffr_10to20 = ffr_10to20[['CONTINENT', 'COUNTRY', 'DOR',
                         'BAS_NAME', 'LENGTH_KM', 'RIV_ORD', 'geometry']]
ffr_20plus = ffr_20plus[['CONTINENT', 'COUNTRY', 'DOR',
                         'BAS_NAME', 'LENGTH_KM', 'RIV_ORD', 'geometry']]

# Set the index to DESIG to remove ramsar sites since we have another better dataset for them
wdpa_africa_polys.set_index('DESIG', inplace=True)
# wdpa_na_polys.set_index('DESIG', inplace=True)
wdpa_sa_polys.set_index('DESIG', inplace=True)
wdpa_asia_polys.set_index('DESIG', inplace=True)
wdpa_europe_polys.set_index('DESIG', inplace=True)
wdpa_australia_polys.set_index('DESIG', inplace=True)


# Dropping passed values
wdpa_africa_polys.drop(
    "Ramsar Site, Wetland of International Importance", inplace=True)

# Set index to NAME
wdpa_africa_polys.set_index('NAME', inplace=True)
# wdpa_na_polys.set_index('NAME', inplace=True)
wdpa_sa_polys.set_index('NAME', inplace=True)
wdpa_asia_polys.set_index('NAME', inplace=True)
wdpa_europe_polys.set_index('NAME', inplace=True)
wdpa_australia_polys.set_index('NAME', inplace=True)

# dropping ALL duplicate values in WDPA dataset based on area Name
wdpa_africa_polys.drop_duplicates(subset=None,
                                  keep='first', inplace=False)
wdpa_sa_polys.drop_duplicates(subset=None,
                                  keep='first', inplace=False)
# wdpa_na_polys.drop_duplicates(subset=None,
#                                   keep='first', inplace=False)
wdpa_asia_polys.drop_duplicates(subset=None,
                                  keep='first', inplace=False)
wdpa_europe_polys.drop_duplicates(subset=None,
                                  keep='first', inplace=False)
wdpa_australia_polys.drop_duplicates(subset=None,
                                  keep='first', inplace=False)

In [ ]:
# Project all data to  World Equidistant Cylindrical, datum WGS84, units meters, EPSG 4087
wdpa_polys_africa = wdpa_africa_polys.to_crs('epsg:4087')
ramsar_polys = ramsar_polys.to_crs('epsg:4087')
proposed_dams = proposed_dams.to_crs('epsg:4087')

ffr_0to5 = ffr_0to5.to_crs('epsg:4087')
ffr_5to10 = ffr_5to10.to_crs('epsg:4087')
ffr_10to20 = ffr_10to20.to_crs('epsg:4087')
ffr_20plus = ffr_20plus.to_crs('epsg:4087')

country_borders = country_borders.to_crs('epsg:4087')
africa = africa_borders.to_crs('epsg:4087')

In [ ]:
# Pull only the ramsar areas for each continent
ramsar_polys_africa = ramsar_polys[ramsar_polys['iso3'].isin(continent_iso[region == Africa])]
ramsar_polys_na = ramsar_polys[ramsar_polys['iso3'].isin(continent_iso[sub-region == North America])]
ramsar_polys_sa = ramsar_polys[ramsar_polys['iso3'].isin(continent_iso[sub-region == South America])]
ramsar_polys_europe = ramsar_polys[ramsar_polys['iso3'].isin(continent_iso[region == Europe])]
ramsar_polys_asia = ramsar_polys[ramsar_polys['iso3'].isin(continent_iso[region == Asia])]
ramsar_polys_aus = ramsar_polys[ramsar_polys['iso3'].isin(continent_iso[region == Oceania])]

# Merge ramsar and WDPA datasets by continent
wdpa_ramsar_africa = pd.concat(wdpa_polys_africa, ramsar_polys_africa)
wdpa_ramsar_na = pd.concat(wdpa_polys_na, ramsar_polys_na)
wdpa_ramsar_sa = pd.concat(wdpa_polys_sa, ramsar_polys_sa)
wdpa_ramsar_euro = pd.concat(wdpa_polys_europe, ramsar_polys_europe)
wdpa_ramsar_as = pd.concat(wdpa_polys_asia, ramsar_polys_asia)
wdpa_ramsar_aus = pd.concat(wdpa_polys_aus, ramsar_polys_aus)

In [ ]:
# Filter for only rivers RIV_ORD < 5
ffr_0to5[ffr_0to5.RIV_ORD < 5]
ffr_5to10[ffr_5to10.RIV_ORD < 5]
ffr_10to20[ffr_10to20.RIV_ORD < 5]
ffr_20plus[ffr_20plus.RIV_ORD < 5]

In [ ]:
# Buffer the FFR to turn them from lines to polys for overlay function
# 1km buffer
ffr_0to5['geometry'] = ffr_0to5.buffer(1000)
ffr_5to10['geometry'] = ffr_5to10.buffer(1000)
ffr_10to20['geometry'] = ffr_10to20.buffer(1000)
ffr_20plus['geometry'] = ffr_20plus.buffer(1000)

# Perform overlay to find only FFR and WDPA sites that overlap eachother
data_intersect_0to5 = gpd.overlay(
    wdpa_ramsar_africa, ffr_0to5, how='intersection')
data_intersect_5to10 = gpd.overlay(
    wdpa_ramsar_africa, ffr_5to10, how='intersection')
data_intersect_10to20 = gpd.overlay(
    wdpa_ramsar_africa, ffr_10to20, how='intersection')
data_intersect_20plus = gpd.overlay(
    wdpa_ramsar_africa, ffr_20plus, how='intersection')

In [ ]:
# Pull only the data for Guinea
bf_border = country_borders[country_borders['CNTRY_NAME'] == "Burkina Faso"]
wdpa_ramsar_bf = wdpa_ramsar_africa[wdpa_ramsar_africa['country_en'] == "Burkina Faso"]
data_intersect_0to5_bf = data_intersect_0to5[data_intersect_0to5['country_en'] == "Burkina Faso"]
data_intersect_5to10_bf = data_intersect_5to10[data_intersect_5to10['country_en'] == "Burkina Faso"]
data_intersect_10to20_bf = data_intersect_10to20[data_intersect_10to20['country_en'] == "Burkina Faso"]
data_intersect_20plus_bf = data_intersect_20plus[data_intersect_20plus['country_en'] == "Burkina Faso"]

In [ ]:
# Create map of FFRs and protected areas
fig, ax = plt.subplots(figsize=(15, 25))
africa.plot(ax=ax, color="lightyellow",
            edgecolor="black", linewidth=2)
wdpa_ramsar_africa.plot(ax=ax, color="lightgreen")
bf_border.plot(ax=ax, facecolor="none",
            edgecolor="black", linewidth=2)
data_intersect_0to5.plot(ax=ax,
                         markersize=15,
                         color='khaki', legend=True)
data_intersect_5to10.plot(ax=ax,
                          markersize=15,
                          color='yellow', legend=True)
data_intersect_10to20.plot(ax=ax,
                           markersize=15,
                           color='orange', legend=True)
data_intersect_20plus.plot(ax=ax,
                           markersize=15,
                           color='red', legend=True)
ctx.add_basemap(ax, url=ctx.providers.Stamen.Terrain, zoom=0)

black_line = mlines.Line2D([], [], color='black', label='Country Border')
khaki_line = mlines.Line2D([], [], color='khaki', label='0 to 5')
yellow_line = mlines.Line2D([], [], color='yellow', label='5 to 10')
orange_line = mlines.Line2D([], [], color='orange', label='10 to 20')
red_line = mlines.Line2D([], [], color='red', label='20 plus')

blue_patch = mpatches.Patch(color='blue', label='Protected Area')

ax.set_title(
    'Figure 1: Portions of Free Flowing Rivers that Overlap Protected Areas\n by Degree of Regulation', size=20)
ax.set_axis_off()
ax.text(0.5, -0.2, "Data Sources: Free Flowing Rivers Dataset\n World Database of Protected Areas\n Ramsar Areas",
        size=12, ha="center", transform=ax.transAxes)
ax.legend(handles=[black_line, khaki_line, yellow_line, orange_line, red_line, blue_patch],
          fontsize=15,
          frameon=True,
          loc=('lower right'),
          title="LEGEND")

In [ ]:
# Create map of FFRs and protected areas
fig, ax = plt.subplots(figsize=(15, 25))
bf_border.plot(ax=ax, color="none",  edgecolor="black", linewidth=2)
wdpa_ramsar_bf.plot(ax=ax, color="lightgreen")
data_intersect_0to5_bf.plot(ax=ax,
                         markersize=15,
                         color='blue', legend=True)
data_intersect_5to10_bf.plot(ax=ax,
                          markersize=15,
                          color='yellow', legend=True)
data_intersect_10to20_bf.plot(ax=ax,
                           markersize=15,
                           color='orange', legend=True)
data_intersect_20plus_bf.plot(ax=ax,
                           markersize=15,
                           color='red', legend=True)
ctx.add_basemap(ax, url=ctx.providers.Stamen.Terrain, zoom=0)

red_patch = mpatches.Patch(color='red', label='Ramsar Area')
green_dot = mlines.Line2D([], [], color='white', marker='o',
                          markerfacecolor='lightgreen', label='Proposed Dam Site')
ax.set_title(
    'Figure 1: Global Proposed Dams & Ramsar Areas', size=20)
ax.set_axis_off()
ax.text(0.5, -0.2, "Data Sources: Global Dam Watch Future Hydropower Reservoirs "
        "and Dams Database (http://globaldamwatch.org/fhred/),\nRamsar Sites "
        "Information Service (https://rsis.ramsar.org/)",
        size=12, ha="center", transform=ax.transAxes)
ax.legend(handles=[red_patch, green_dot, black_line],
          fontsize=15,
          frameon=True,
          loc=('lower right'),
          title="LEGEND")